In [1]:
import pandas as pd
import re

In [2]:
qa_df = pd.read_csv('docx/qa/qa-chinese.csv')
qa_df.head()

,summary,content,summary_token_num,content_token_num,token_num,context,questions,answers
0,简介,中国联通手机营业厅分为网页版和客户端，提供7×24小时服务。目前功能包括：话费查询、交费充值...,1,69,70,summary: 简介\ncontent: 中国联通手机营业厅分为网页版和客户端，提供7×2...,1. 中国联通手机营业厅分为网页版和客户端吗?\n2. 客户端提供7×24小时服务吗?\n3...,1. 中国联通手机营业厅分为网页版和客户端。\n2. 客户端提供7×24小时服务。\n3. ...
1,访问方式#网页版,中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时...,6,60,66,summary: 访问方式#网页版\ncontent: 中国联通手机营业厅网页版wap.10...,1. 中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×2...,1. 中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×2...
2,访问方式#客户端,下载方式：手机登陆wap.10010.com下载手机登录www.10010.com下载目前，...,5,96,101,summary: 访问方式#客户端\ncontent: 下载方式：手机登陆wap.10010...,1. 下载手机登录www.10010.com下载的用户有哪些?\n2. 不同手机品牌、不同用...,1. 下载手机登录www.10010.com下载的用户有iPhone版、iPad版、Andr...
3,常用功能#客户端各功能,当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，...,7,312,319,summary: 常用功能#客户端各功能\ncontent: 当月话费用户登录客户端后，点击...,1. 当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户...,1. 当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户...
4,常用功能#手机营业厅网页版功能列表,网页版手机营业厅wap.10010.com常用功能1、话费查询话费查询目前可提供当月话费、可...,10,1110,1120,summary: 常用功能#手机营业厅网页版功能列表\ncontent: 网页版手机营业厅w...,1.,1.


In [3]:
question_list, answer_list = [], []
for idx, row in qa_df.iterrows():
    questions = row['questions'].split('\n')
    answers = row['answers'].split('\n')
    min_len = min(len(questions), len(answers))
    question_list.extend(list(map(lambda x: re.sub(r"^[0-9]+\.", "", x).strip(" \""), questions[:min_len])))
    answer_list.extend(list(map(lambda x: re.sub(r"^[0-9]+\.", "", x).strip(" \""), answers[:min_len])))

new_qa_df = pd.DataFrame({'questions': question_list, 'answers': answer_list})

new_qa_df.to_csv('docx/qa/qa-chinese-new.csv', index=False)
new_qa_df.head(10)

,questions,answers
0,中国联通手机营业厅分为网页版和客户端吗?,中国联通手机营业厅分为网页版和客户端。
1,客户端提供7×24小时服务吗?,客户端提供7×24小时服务。
2,目前功能包括哪些?,目前功能包括话费查询、交费充值、3G套餐变更、业务查询、积分查询、密码修改、积分兑换、网点查...
3,业务查询功能包括哪些?,业务查询
4,中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时...,中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时...
5,您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联...,您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联...
6,下载手机登录www.10010.com下载的用户有哪些?,下载手机登录www.10010.com下载的用户有iPhone版、iPad版、Android...
7,不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使...,不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使...
8,当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，...,当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，...
9,,


In [4]:
import opencc

converter = opencc.OpenCC('t2s.json')

def postprocess(df, min_len=10, max_len=300):
    print(f"Before postprocessing: {len(df)}")
    # Remove too short or too long sentences
    df = df[(df['questions'].str.len() > min_len) & (df['questions'].str.len() < max_len)]
    df = df[(df['answers'].str.len() > min_len) & (df['answers'].str.len() < max_len)]
    print(f"After length filtering: {len(df)}")
    # Remove questions not ending with question mark
    df = df[df['questions'].str.endswith('?')]
    print(f"After question mark filtering: {len(df)}")
    # Remove answers not ending with period
    df = df[df['answers'].str.endswith('。')]
    print(f"After answer filtering: {len(df)}")
    # Remove duplicated sentences
    df = df.drop_duplicates(subset=['questions', 'answers'])
    print(f"After duplicate filtering: {len(df)}")

    df['questions'] = df['questions'].apply(lambda x: converter.convert(x))
    df['answers'] = df['answers'].apply(lambda x: converter.convert(x))

    return df

filtered_qa_df = postprocess(new_qa_df)
filtered_qa_df.to_csv('docx/qa/qa-chinese-filtered.csv', index=False)
filtered_qa_df.head(10)

Before postprocessing: 700
After length filtering: 530
After question mark filtering: 430
After answer filtering: 307
After duplicate filtering: 307


,questions,answers
0,中国联通手机营业厅分为网页版和客户端吗?,中国联通手机营业厅分为网页版和客户端。
1,客户端提供7×24小时服务吗?,客户端提供7×24小时服务。
4,中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时...,中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时...
5,您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联...,您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联...
6,下载手机登录www.10010.com下载的用户有哪些?,下载手机登录www.10010.com下载的用户有iPhone版、iPad版、Android...
7,不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使...,不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使...
13,中国联通采用的是什么标准?,中国联通采用的是通信标准。
14,"与其它标准相比,全球开通网络最多是什么样的?",全球开通网络最多是采用制式的。
15,全球有多少个国家具有网络覆盖全球市场?,全球有个国家具有网络覆盖全球市场。
16,全球市场占有率最高的终端款式是什么?,全球市场占有率最高的终端款式是支持商用终端的款式。


In [7]:
import opencc

# converter = opencc.OpenCC("s2t.json") # simplified to traditional
# converter.convert("汉字")

converter = opencc.OpenCC("t2s.json") # traditional to simplified
converter.convert("可通过網上营业厅、客服熱線或携帶身份證营业厅办理停/開机, this is english ")

'可通过网上营业厅、客服热线或携带身份证营业厅办理停/开机, this is english '